## **Smart Health Guide**

In [11]:
!pip install openai
!pip install langchain

In [12]:
!pip -q install langchain tiktoken duckduckgo-search

In [13]:
!pip install gradio

In [14]:
import gradio as gr
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun
from langchain.memory import ConversationBufferWindowMemory
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-Sg97wUMr2h0jvgqgBCP4T3BlbkFJoM1vtOPaila3KK4QY0Iq"

The code below initializes a DuckDuckGo search tool named "General Health Search," which leverages DuckDuckGo's search functionality to respond to health-related questions. The tool is designed to provide up-to-date medical information and answer general health queries.

In [15]:
# Defining the tools for the Smart Health Guide
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="General Health Search",
        func=search.run,
        description="Use this tool to search general health questions and recent medical information."
    )
]

Defining a custom prompt template class for agents, enabling the dynamic construction of prompts with relevant information. It formats input by incorporating a scratchpad of previous agent actions and tool observations, along with listing available tools and their descriptions. This structured prompt supports more effective decision-making in agents, providing them with a detailed context for processing tasks.

In [16]:
# Implementing a Custom Prompt Template class
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

The below cell provides a structured template for a health guide agent to follow when processing user inquiries, listing available tools and guiding the agent step-by-step to generate a comprehensive and empathetic response.

In [17]:
# Prompt Template
template = """As a knowledgeable Smart Health Guide, respond empathetically to the user's health concerns. The available tools are:

{tools}

Use this structure:

User Inquiry: the input question to answer
Thought: carefully consider the next best step
Action: choose from [{tool_names}]
Action Input: input for the action
Observation: result of the action
... (repeat N times if necessary)
Thought: I now know the final answer
Final Answer: the final answer to the original inquiry

Begin as a compassionate health guide.

User Inquiry: {input}
{agent_scratchpad}"""

The code defines a custom output parser class that interprets responses generated by an LLM to determine the agent's next step. If a "Final Answer" is found, it concludes the process and provides the final response. Otherwise, it searches the response for the next "Action" and "Action Input" using regular expressions, returning the corresponding action or an error message if no valid action is detected.

In [18]:
# Custom Output Parser
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Final Answer:" in llm_output:
            return AgentFinish(return_values={"output": llm_output.split("Final Answer:")[-1].strip()}, log=llm_output)

        regex = r"Action\s*:\s*(.*?)\nAction\s*Input\s*:\s*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            return AgentFinish(
                return_values={"output": "I'm sorry, I couldn't determine the appropriate action. Please consult a healthcare provider for accurate information or rephrase your question."},
                log=llm_output
            )

        action = match.group(1).strip()
        action_input = match.group(2).strip()
        return AgentAction(tool=action, tool_input=action_input.strip('"'), log=llm_output)


We set up an AI agent that leverages a custom prompt template and output parser to handle health-related user queries. It initializes an OpenAI language model chain using the custom prompt, while the output parser ensures the correct interpretation of LLM responses. The agent, built with a single-action logic, is configured with specific tools it can utilize, and an executor with a memory buffer is created to maintain a context-aware conversation, remembering up to 10 past interactions for more consistent responses.

In [19]:
# Creating an LLM Chain
prompt = CustomPromptTemplate(template=template, tools=tools, input_variables=["input", "intermediate_steps"])
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Create Agent
output_parser = CustomOutputParser()
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

# Agent Executor with Memory
memory = ConversationBufferWindowMemory(k=10)
agent_executor_mem = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMSingleActionAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.

The health_guide function takes user inputs (query, age, gender, and symptoms) and builds a detailed query by adding this contextual information. It then passes this information to the agent executor, which processes the query using the pre-configured agent logic and returns the response. If an error occurs during processing, it catches the exception and returns an error message advising the user to clarify their question or consult a healthcare provider.

In [20]:
# Gradio Function
def health_guide(query, age, gender, symptoms):
    try:
        # Including additional input details in the query for context
        context_info = f"Age: {age}, Gender: {gender}, Symptoms: {symptoms}\n"
        return agent_executor_mem.run(input=context_info + query)
    except Exception as e:
        return f"Error: {e}. Please ensure your question is clear, or consult a healthcare provider for guidance."

It defines the input fields with a text box for the health question and symptoms, a slider for age, and a dropdown menu for gender. The output is displayed in a text box labeled "Smart Health Guide Response," and the interface includes a disclaimer to consult a healthcare provider. The live mode ensures real-time updates, and share=True makes the interface publicly accessible while debug=True helps identify issues during use.

In [21]:
# Updated Gradio Interface
interface = gr.Interface(
    fn=health_guide,
    inputs=[
        gr.Textbox(lines=2, placeholder="Ask your health question here...", label="Health Question"),
        gr.Slider(0, 120, step=1, label="Age"),
        gr.Dropdown(choices=["Male", "Female", "Non-binary", "Other"], label="Gender"),
        gr.Textbox(lines=2, placeholder="Describe your symptoms here...", label="Symptoms")
    ],
    outputs=gr.Textbox(label="Smart Health Guide Response"),
    title="Smart Health Guide",
    description="""Disclaimer: This guide is for informational purposes only.
                    Always consult a healthcare provider for proper diagnosis and treatment.""",
    live=True
)

# Launch Gradio Interface
interface.launch( debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://537064d744a82b6006.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: It's understandable that you're concerned about your health. Let's see if we can figure out what might be causing your symptoms.
Action: General Health Search
Action Input: "Possible causes of fatigue and headaches"

Observation:A headache and fatigue are often a symptom of something else — and it can be hard to narrow down or pinpoint the exact cause. "Headache and fatigue are two different symptoms," says Dr. Vyas. Allergies, Hay Fever, and Fatigue. Symptoms: Fatigue, headache, itchiness, nasal congestion, and drainage. Allergic rhinitis is a common cause of chronic fatigue. But allergic rhinitis often can ... Stress. Stress can cause tight muscles in the shoulders and neck, which often leads to tension headaches. When tension headaches become frequent, the pain in shoulder and neck muscles is felt by the brain as pain in the head. Hunger itself can trigger a migraine or tension headache. But eating certain foods may trigger migraines.